# Time series classification with scikit-time

The Time Series Classification (TSC) task involves training a model from a collection
 of time series (real valued, ordered, data) in order to predict a target variable.
 For example, we might want to build a model that can predict whether a patient is
 sick based on the ECG reading, or predict whether a device will fail based on some
 sensor reading. This notebook gives a quick guide to get you started. Some basic
 terminology
1. A time series: an ordered, real valued sequence of observations.
2. target/class label:  an observation of the variable we want to be able to predict.
3. an instance or case: a time series and its associated target label.
4. A data set: a collection of instances


<img src="./img/tsc.png" width="600" alt="time series classification"> [<i>&#x200B;</i>](./img/tsc.png)

## Problem types and data structures
Time series are either univariate (a time series has a single value at
each time point) or multivariate (a time series has multiple values at each time
point). Series can be the same length for all instances, or may be of different
lengths. Series may contain missing values. When series are equal length, we recommend
storing the time series in numpy arrays. For example, for a problem with
10 time series with 1 dimension (univariate) and series length `50, we can store
the series in a 2D numpy array.  2D numpy for equal length univariate problems will work with all classifiers. It
also directly mirrors scikit-learn, which will treat each time point as an attribute.

In [ ]:
import numpy as np

# univariate set of 10 time series of length 50
# Each row is a time series.
train_uni = np.random.rand(10,50)
# Random train labels for a two class problem
train_labels =  np.random.randint(low=0, high=2, size=10)
# You can use standard sckit classifiers with data in this format, where each time
# point is treated as an attribute
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(train_uni, train_labels)
pred = rf.predict(test_uni)

However, we recommend always using 3D numpy arrays because other scikit-time estimators
 may interpret 2D numpy arrays as a single multidimensional time series of shape
 (length, n_dimension).

In [ ]:
train_uni=train_uni.reshape(train_uni.shape[0], 1, train_uni.shape[1])
# a random multivariate set of 10 time series with 3 dimensions of length 50
train_multi = np.random.rand(10, 3, 50)

If the time series are not of equal length, we cannot use numpy arrays, since they do
 not support ragged arrays. In this case, we currently use DataFrames, where each row
is an instance, each column is a dimension. A cell in this format is a univariate
time series in pd.Series format.


## Datasets and Problem Types


The UCR/UEA [TSC dataset archive]
(https://timeseriesclassification.com/)
contains a large number of example TSC problems that have been used thousands of
times in the literature to assess TSC algorithms. Most datasets in
the archive contain time series all the same length and all are assumed to be sampled
 at the same frequency, and have no missing values.

[ArrowHead dataset](https://timeseriesclassification.com/description.php?Dataset=ArrowHead) consists of outlines of the images of arrow heads. The classification of projectile points is an important topic in anthropology.

<img src="./img/arrow-heads.png" width="600" alt="arrow heads">

The shapes of the projectile points are converted into a sequence using the angle-based method as described in this [blog post](https://izbicki.me/blog/converting-images-into-time-series-for-data-mining.html) about converting images into time series for data mining.

<img src="./img/from-shapes-to-time-series.png" width="600" alt="from shapes to time series">

Each instance consists of a single time series (i.e. the problem is univariate) of equal length and a class label based on shape distinctions such as the presence and location of a notch in the arrow. The data set consists of 210 instances, by default split into 36 train and 175 test instances. We refer to the collection of time series as $X$ and to the collection of class labels as $y$.

Below, we store the data in a 3D dimensional (instance, dimension, time point) numpy
array for $X$, and a one dimensional numpy array for $y$.

For the single problem loader load arrow head, set the return type to `numpy3D` to
store $X$ in such a 3D ndarray or `numpy2d` (2D numpy with rows=instances,
columns=time points; alias is `numpyflat`).

In [ ]:
# Plotting and data loading imports used in this notebook
import matplotlib.pyplot as plt

from sktime.datasets import (
    load_arrow_head,
    load_basic_motions,
    load_japanese_vowels,
    load_plaid,
)

In [ ]:
# Load all arrow head
arrow_X, arrow_y = load_arrow_head(return_type="numpy3d")
# Load default train/test splits from sktime/datasets/data
arrow_train_X, arrow_train_y = load_arrow_head(split="train", return_type="numpy3d")
arrow_test_X, arrow_test_y = load_arrow_head(split="test", return_type="numpy3d")
print(arrow_train_X.shape, arrow_train_y.shape, arrow_test_X.shape, arrow_test_y.shape)
plt.title("First instance in ArrowHead data")
plt.plot(arrow_train_X[0, 0])

In [ ]:
# Load arrow head dataset in nested pandas format, also accepted by sktime classifiers
arrow_train_X, arrow_train_y = load_arrow_head(split="train", return_type="nested_univ")
arrow_test_X, arrow_test_y = load_arrow_head(split="test", return_type="nested_univ")
arrow_train_X.iloc[:5]

In [ ]:
# Load arrow head dataset in numpy2d format, also accepted by sktime classifiers
arrow_train_X, arrow_train_y = load_arrow_head(split="train", return_type="numpy2d")
arrow_test_X, arrow_test_y = load_arrow_head(split="test", return_type="numpy2d")
print(arrow_train_X.shape, arrow_train_y.shape, arrow_test_X.shape, arrow_test_y.shape)

Some TSC datasets are multivariate, in that each time series instance has more than one variable. For example, the [BasicMotions dataset](https://timeseriesclassification.com/description.php?Dataset=BasicMotions) was generated as part of a student project where four students performed four activities whilst wearing a smartwatch. The watch collects 3D accelerometer and 3D gyroscope data. Each instance involved a subject performing one of four tasks (walking, resting, running and badminton) for ten seconds. Time series in this data set have 6 variables.

In [ ]:
# "basic motions" dataset
motions_X, motions_Y = load_basic_motions(return_type="numpy3d")
motions_train_X, motions_train_y = load_basic_motions(
    split="train", return_type="numpy3d"
)
motions_test_X, motions_test_y = load_basic_motions(split="test", return_type="numpy3d")
print(type(motions_train_X))
print(
    motions_train_X.shape,
    motions_train_y.shape,
    motions_test_X.shape,
    motions_test_y.shape,
)
plt.title(" First and second dimensions of the first instance in BasicMotions data")
plt.plot(motions_train_X[0][0])
plt.plot(motions_train_X[0][1])

Some data sets have unequal length series. Two data sets with this characteristic are
 shipped with scikit-time: PLAID (univariate) and JapaneseVowels (multivariate). We
 cannot store unequal length series in `numpy` arrays. Instead, we use a nested `pandas` `DataFrame`, where each cell is a `pandas` `Series`. This is the default return type for all single problem loaders.

In [ ]:
# loads both train and test together
vowel_X, vowel_y = load_japanese_vowels()
print(type(vowel_X))

plt.title(" First two dimensions of two instances of Japanese vowels")
plt.plot(vowel_X.iloc[0, 0], color="b")
plt.plot(vowel_X.iloc[1, 0], color="b")
plt.plot(vowel_X.iloc[0, 1], color="r")
plt.plot(vowel_X.iloc[1, 1], color="r")
plt.show()

In [ ]:
plaid_X, plaid_y = load_plaid()
plaid_train_X, plaid_train_y = load_plaid(split="train")
plaid_test_X, plaid_test_y = load_plaid(split="test")
print(type(plaid_X))

plt.title(" Four instances of PLAID dataset")
plt.plot(plaid_X.iloc[0, 0])
plt.plot(plaid_X.iloc[1, 0])
plt.plot(plaid_X.iloc[2, 0])
plt.plot(plaid_X.iloc[3, 0])
plt.show()

## Building Classifiers

We show the simplest use cases for classifiers and demonstrate how to build bespoke pipelines for time series classification.

It is possible to use a standard `sklearn` classifier for univariate, equal length classification problems, but it is unlikely to perform as well as bespoke time series classifiers, since supervised tabular classifiers ignore the sequence information in the variables.

To apply `sklearn` classifiers directly, the data needs to be reshaped into one of the sklearn compatible 2D data formats. `sklearn` cannot be used directly with multivariate or unequal length data sets, without making choices in how to insert the data into a 2D structure.

`scikit-time` provides functionality to make these choices explicit and tunable,
under a
unified interface for time series classifiers.

`scikit-time` also provides pipeline construction functionality for transformers and classifiers that are specific to time series datasets.

Direct application of `sklearn` (without `scikit-time`) is possible via using the `numpy2d` return type for the time series data sets, and then feeding the format into `sklearn`:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

classifier = RandomForestClassifier(n_estimators=100)
arrow_train_X_2d, arrow_train_y_2d = load_arrow_head(
    split="train", return_type="numpy2d"
)
arrow_test_X_2d, arrow_test_y_2d = load_arrow_head(split="test", return_type="numpy2d")
classifier.fit(arrow_train_X_2d, arrow_train_y_2d)
y_pred = classifier.predict(arrow_test_X_2d)

accuracy_score(arrow_test_y_2d, y_pred)

`scikit-time` contains the state of the art in time series classifiers in the package classification. These are grouped based on their representation. An accurate and relatively fast classifier is the [ROCKET](https://link.springer.com/article/10.1007/s10618-020-00701-z) algorithm:

In [ ]:
from sktime.classification.kernel_based import RocketClassifier

rocket = RocketClassifier(num_kernels=2000)
rocket.fit(arrow_train_X, arrow_train_y)
y_pred = rocket.predict(arrow_test_X)

accuracy_score(arrow_test_y, y_pred)

Another accurate classifier for time series classification is version 2 of the [HIVE-COTE](https://link.springer.com/article/10.1007/s10994-021-06057-9) algorithm. HC2 is slow on small problems like these examples. However, it can be configured with an approximate maximum run time as follows (may take a bit longer than 12 seconds to run this cell):

In [ ]:
from sktime.classification.hybrid import HIVECOTEV2

hc2 = HIVECOTEV2(time_limit_in_minutes=0.2)
hc2.fit(arrow_train_X, arrow_train_y)
y_pred = hc2.predict(arrow_test_X)

accuracy_score(arrow_test_y, y_pred)

## sklearn Compatibility

`scikit-time` classifiers are compatible with `sklearn` model selection and composition
tools using `sktime` data formats. We provide some functionality examples in the following.

Cross-validation using the `sklearn` `cross_val_score` and `KFold` functionality:

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

cross_val_score(rocket, arrow_train_X, y=arrow_train_y, cv=KFold(n_splits=4))

Parameter tuning using `sklearn` `GridSearchCV`, we tune the _k_ and distance measure for a K-NN classifier:

In [ ]:
from sklearn.model_selection import GridSearchCV

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

knn = KNeighborsTimeSeriesClassifier()
param_grid = {"n_neighbors": [1, 5], "distance": ["euclidean", "dtw"]}
parameter_tuning_method = GridSearchCV(knn, param_grid, cv=KFold(n_splits=4))

parameter_tuning_method.fit(arrow_train_X, arrow_train_y)
y_pred = parameter_tuning_method.predict(arrow_test_X)

accuracy_score(arrow_test_y, y_pred)

Probability calibration with the `sklearn` `CalibratedClassifierCV`:

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

from sktime.classification.interval_based import DrCIF

calibrated_drcif = CalibratedClassifierCV(
    base_estimator=DrCIF(n_estimators=10, n_intervals=5), cv=4
)

calibrated_drcif.fit(arrow_train_X, arrow_train_y)
y_pred = calibrated_drcif.predict(arrow_test_X)

accuracy_score(arrow_test_y, y_pred)

## Multivariate Classification

Many classifiers, including ROCKET and HC2, are configured to work with multivariate input. For example:

In [ ]:
from sktime.classification.kernel_based import RocketClassifier

rocket = RocketClassifier(num_kernels=2000)
rocket.fit(motions_train_X, motions_train_y)
y_pred = rocket.predict(motions_test_X)

accuracy_score(motions_test_y, y_pred)

In [ ]:
from sktime.classification.hybrid import HIVECOTEV2

HIVECOTEV2(time_limit_in_minutes=0.2)
hc2.fit(motions_train_X, motions_train_y)
y_pred = hc2.predict(motions_test_X)

accuracy_score(motions_test_y, y_pred)

## Classification with Unequal Length Series

A common trait in time series data is absence of a uniform series length, as seen in the PLAID and JapaneseVowels datasets introduced previously. None of the `numpy` data formats support ragged arrays, as such one of the `pandas` `DataFrame` formats must be used for unequal length data.

At the time of writing the number of classifiers which natively support unequal length series is limited. The following outputs the current classifiers which support unequal length data.

In [ ]:
from sktime.registry import all_estimators

# search for all classifiers which can handle unequal length data. This may give some
# UserWarnings if soft dependencies are not installed.
all_estimators(
    filter_tags={"capability:unequal_length": True}, estimator_types="classifier"
)

Certain `scikit-time` transformers such as the `PaddingTransformer` and
`TruncationTransformer`
 can be used in a pipeline to process unequal length data for use in a wider range of classification algorithms.

### Background info and references for classifiers used here

#### KNeighborsTimeSeriesClassifier

One nearest neighbour (1-NN) classification with Dynamic Time Warping (DTW) is one of the oldest TSC approaches, and is commonly used as a performance benchmark.

#### RocketClassifier
The RocketClassifier is based on a pipeline combination of the ROCKET transformation (transformations.panel.rocket) and the sklearn RidgeClassifierCV classifier. The RocketClassifier is configurable to use variants MiniRocket and MultiRocket. ROCKET is based on generating random convolutional kernels. A large number are generated, then a linear classifier is built on the output.

[1] Dempster, Angus, François Petitjean, and Geoffrey I. Webb. "Rocket: exceptionally fast and accurate time series classification using random convolutional kernels." Data Mining and Knowledge Discovery (2020)
[arXiv version](https://arxiv.org/abs/1910.13051)
[DAMI 2020](https://link.springer.com/article/10.1007/s10618-020-00701-z)

#### DrCIF
The Diverse Representation Canonical Interval Forest Classifier (DrCIF) is an interval based classifier. The algorithm takes multiple randomised intervals from each series and extracts a range of features. These features are used to build a decision tree, which in turn are ensembled into a decision tree forest, in the style of a random forest.

Original CIF classifier:
[2] Matthew Middlehurst and James Large and Anthony Bagnall. "The Canonical Interval Forest (CIF) Classifier for Time Series Classification." IEEE International Conference on Big Data (2020)
[arXiv version](https://arxiv.org/abs/2008.09172)
[IEEE BigData (2020)](https://ieeexplore.ieee.org/abstract/document/9378424?casa_token=8g_IG5MLJZ4AAAAA:ItxW0bY4eCRwfdV9kLvf-8a8X73UFCYUGU9D19PwrHigjivLJVchxHwkM3Btn7vvlOJ_0HiLRa3LCA)

The DrCIF adjustment was proposed in [3].

#### HIVE-COTE 2.0 (HC2)
The HIerarchical VotE Collective of Transformation-based Ensembles is a meta ensemble that combines classifiers built on different representations. Version 2  combines DrCIF, TDE, an ensemble of RocketClassifiers called the Arsenal and the  ShapeletTransformClassifier. It is one of the most accurate classifiers on the UCR and UEA time series archives.

[3] Middlehurst, Matthew, James Large, Michael Flynn, Jason Lines, Aaron Bostrom, and Anthony Bagnall. "HIVE-COTE 2.0: a new meta ensemble for time series classification." Machine Learning (2021)
[ML 2021](https://link.springer.com/article/10.1007/s10994-021-06057-9)

#### Catch22

The CAnonical Time-series CHaracteristics (Catch22) are a set of 22 informative and low redundancy features extracted from time series data. The features were filtered from 4791 features in the `hctsa` toolkit.

[4] Lubba, Carl H., Sarab S. Sethi, Philip Knaute, Simon R. Schultz, Ben D. Fulcher, and Nick S. Jones. "catch22: Canonical time-series characteristics." Data Mining and Knowledge Discovery (2019)
[DAMI 2019](https://link.springer.com/article/10.1007/s10618-019-00647-x)